In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import one_hot
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
# import datamol as dm
# import rdkit
# from rdkit import Chem
# from rdkit.Chem import rdFingerprintGenerator
# import useful_rdkit_utils as uru
import sys
from matplotlib import pyplot as plt
print(f"Pandas version used: {pd.__version__}")
print(f"PyTorch version used: {torch.__version__}")
print(f"NumPy version used: {np.__version__}")
#print(f"RDKit version used: {rdkit.__version__}")
print(f"Python version used: {sys.version}")

Pandas version used: 2.2.3
PyTorch version used: 2.2.2
NumPy version used: 1.26.4
Python version used: 3.12.7 (main, Oct 16 2024, 09:10:10) [Clang 18.1.8 ]


In [2]:
# PyTorch example re. saving & reloading tensors
t = torch.tensor([1., 2.])
torch.save(t, 'tensor.pt')
ts = torch.load('tensor.pt')
ts

tensor([1., 2.])

In [3]:
# Load adrs tensors from 2_ADR_regressor.ipynb after it's saved (from 2_ADR_regressor_save_tensors.ipynb)
adrs_ts = torch.load("adr_train_tensors.pt")
adrs_ts

tensor([[-1.5256, -0.7502],
        [-0.6540, -1.6095],
        [-0.1002, -0.6092],
        ...,
        [ 0.8748,  0.9873],
        [-0.7102,  2.8641],
        [ 1.1651,  2.0154]], requires_grad=True)

In [ ]:
# Plan
# If wanting to use tokenizer.decode(), likely may need to build a tokenization/tokenizer model first...?

# consider trying HuggingFace's transformers:
# 1. Set up tokenizer model that will tokenize the ADRs/words
# 2. Apply tokenizer.decode() function to each tensor row/sequence (via using list comprehension)
# 3. Use sample code snippet below to decode tensors: 
# decoded = [tokenizer.decode(x) for x in adrs_ts]
# the code will iterate through each row/sequence of tensors and apply the decode() method 
# which'll transform the numerical IDs back into human-readable texts/words

# other methods that may be useful:
# convert_ids_to_tokens - converts numerical IDs back into corresponding token identifiers
# convert_tokens_to_string - merges sub-word tokens into complete words if needed

In [5]:
# may need to also use/integrate code with tokenizers package? (https://pypi.org/project/tokenizers/)